## Deliverable 2. Create a Customer Travel Destinations Map.

In [7]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [8]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Severo-Kurilsk,RU,50.6789,156.1250,11.24,73,95,1.14,overcast clouds
1,1,Lai,TD,9.3952,16.2989,22.82,86,100,2.54,overcast clouds
2,2,Haimen,CN,23.1935,116.6142,27.94,80,100,5.59,light rain
3,3,Qaanaaq,GL,77.4840,-69.3632,-0.12,77,38,2.24,scattered clouds
4,4,Iqaluit,CA,63.7506,-68.5145,0.85,80,75,5.66,broken clouds


In [9]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input('What is the minimum temperature you would like for your trip (in C)?  '))

max_temp = float(input('What is the maximum temperature you would like for your trip (in C)?  '))

What is the minimum temperature you would like for your trip (in C)?  24
What is the maximum temperature you would like for your trip (in C)?  32


In [13]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
2,2,Haimen,CN,23.1935,116.6142,27.94,80,100,5.59,light rain
8,8,Russell,US,32.3502,-85.1999,29.78,94,0,0.00,clear sky
16,16,Avarua,CK,-21.2078,-159.7750,25.03,73,57,2.06,broken clouds
17,17,Kapaa,US,22.0752,-159.3190,26.99,86,100,5.14,overcast clouds
22,22,Liuzhou,CN,24.3126,109.3892,27.04,87,100,3.58,overcast clouds
27,27,Port Hedland,AU,-20.3167,118.5667,30.03,21,75,6.17,broken clouds
31,31,Vaini,TO,-21.2000,-175.2000,26.09,78,40,4.12,scattered clouds
35,35,Cockburn Town,TC,21.4612,-71.1419,27.48,80,77,6.70,broken clouds
39,39,Georgetown,MY,5.4112,100.3354,30.97,74,40,3.60,scattered clouds
43,43,Hithadhoo,MV,-0.6000,73.0833,27.71,75,99,4.87,overcast clouds


In [11]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                210
City                   210
Country                210
Lat                    210
Lng                    210
Max Temp               210
Humidity               210
Cloudiness             210
Wind Speed             210
Current Description    210
dtype: int64

In [15]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.

clean_df = preferred_cities_df.dropna(axis=0, how='any')
clean_df.count()

# There were no missing values to begin w so none are dropped

City_ID                210
City                   210
Country                210
Lat                    210
Lng                    210
Max Temp               210
Humidity               210
Cloudiness             210
Wind Speed             210
Current Description    210
dtype: int64

In [16]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,Haimen,CN,27.94,light rain,23.1935,116.6142,
8,Russell,US,29.78,clear sky,32.3502,-85.1999,
16,Avarua,CK,25.03,broken clouds,-21.2078,-159.7750,
17,Kapaa,US,26.99,overcast clouds,22.0752,-159.3190,
22,Liuzhou,CN,27.04,overcast clouds,24.3126,109.3892,
27,Port Hedland,AU,30.03,broken clouds,-20.3167,118.5667,
31,Vaini,TO,26.09,scattered clouds,-21.2000,-175.2000,
35,Cockburn Town,TC,27.48,broken clouds,21.4612,-71.1419,
39,Georgetown,MY,30.97,scattered clouds,5.4112,100.3354,
43,Hithadhoo,MV,27.71,overcast clouds,-0.6000,73.0833,


In [25]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = 'https://maps.googleapis.com/maps/api/place/nearbysearch/json'

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, 'Hotel Name'] = hotels['results'][0]['name']
    except (IndexError):
        print('Hotel not found... skipping.')
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [36]:
# 7. Drop the rows where there is no Hotel Name.
print(f'{hotel_df.isnull().sum()}')

# no missing values present but we can drop na just in case

hotel_df = hotel_df.dropna(axis=0, how='any')
hotel_df.count()



City                   0
Country                0
Max Temp               0
Current Description    0
Lat                    0
Lng                    0
Hotel Name             0
dtype: int64


City                   210
Country                210
Max Temp               210
Current Description    210
Lat                    210
Lng                    210
Hotel Name             210
dtype: int64

In [40]:
# 8a. Create the output File (CSV)
output_data_file = '../Vacation_Search/WeatherPy_vacation.csv'
# 8b. Export the City_Data into a csv
hotel_df.to_csv(output_data_file,index_label='City_ID')

hotel_df.head()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,Haimen,CN,27.94,light rain,23.1935,116.6142,Shantou Chaoyang Donghui Hotel
8,Russell,US,29.78,clear sky,32.3502,-85.1999,Macs
16,Avarua,CK,25.03,broken clouds,-21.2078,-159.7750,Paradise Inn
17,Kapaa,US,26.99,overcast clouds,22.0752,-159.3190,Sheraton Kauai Coconut Beach Resort
22,Liuzhou,CN,27.04,overcast clouds,24.3126,109.3892,Linjiang Inn


In [38]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °C</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]

In [39]:
# 11a. Add a marker layer for each city to the map. 

fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# 11b. Display the figure

fig

Figure(layout=FigureLayout(height='420px'))